In [1]:
# Dataframes
import numpy as np
import pandas as pd
from copy import deepcopy

# Plots
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,8)
plt.style.use('ggplot')

# Scikit Learn
from sklearn.cross_validation  import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import preprocessing


from scipy.stats import multivariate_normal
from scipy.stats import mode

C:\Users\prash\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Path of the folder
path = 'C:/Users/prash/Downloads/ML ALGORITHMS/'

# Number of clusters required
k = 3

max_iter = 10

In [3]:
# Import Iris Dataset
iris_dataset = pd.read_csv(path + 'DATASETS/' + 'Iris.csv')
# Removing Index Column
iris_dataset = iris_dataset.iloc[:,1:]

# Input Dataframe
X = iris_dataset.iloc[:,:-1]
X = np.array(X)

# Normalize the Dataframe
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

# Encode the Output labels
Y = iris_dataset.iloc[:,-1]
for i in range(len(Y.unique())):
    Y = Y.replace(Y.unique()[i],i)
Y = np.array(Y)

# Divide into train and test datasets
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [4]:
def predict_proba(X,mu,sigma,phi,k):
    
    n, m = X.shape
    
    # Calculate Gaussian Distributions and Likelihoods for all the classes
    likelihood = np.zeros((n,k))
    for i in range(k):
        distribution = multivariate_normal(mean = mu[i], cov = sigma[i])
        likelihood[:,i] = distribution.pdf(X)
    
    # Probability of Each Class
    numerator = likelihood * phi
    # Sum of all Probabilities
    denominator = numerator.sum(axis=1)[:, np.newaxis]
    
    weights = numerator / denominator
    
    return weights

def e_step(X,mu,sigma,phi,k):
    weights = predict_proba(X,mu,sigma,phi,k)
    phi = weights.mean(axis=0)
    
    return weights,phi
    
def m_step(X,weights):
    for i in range(k):
        weight = weights[:, [i]]
        total_weight = weight.sum()
        
        mu[i] = (X * weight).sum(axis=0) / total_weight
        sigma[i] = np.cov(X.T, aweights=(weight/total_weight).flatten(), bias=True)
        
        return mu,sigma

In [5]:
# Initialize the Parameters

n, m = X_train.shape

phi = np.full(shape = k, fill_value = 1/k)

random_row = np.random.randint(low = 0, high = n, size = k)
mu = [X_train[row_index,:] for row_index in random_row]
sigma = [np.cov(X_train.T) for _ in range(k)]

# Train

for iteration in range(max_iter):
    weights,phi = e_step(X_train,mu,sigma,phi,k)
    mu,sigma = m_step(X_train,weights)
    
print("The trained parameters are:")
print("\nPhi:\n\n",phi)
print("\nmu:\n\n",mu)
print("\nsigma:\n\n",sigma)
    
# Test
    
weights = predict_proba(X_test,mu,sigma,phi,k)
Y_pred = np.argmax(weights, axis=1)

# Make sure that the clusters represented in both Y_test and Y_pred are the same
permutation = []
for i in range(k):
    dummy = Y_test[Y_pred == i]
    if(dummy.size > 0):
        permutation.append(mode(dummy).mode.item())
        
    # Not sure about this
    else:
        permutation.append(0)
permutation = np.array(permutation)
permuted_prediction = permutation[Y_pred]

# Compute test set accuracy  
acc = accuracy_score(permuted_prediction, Y_test)
print("\nTest set accuracy: {:.2f}".format(acc))

The trained parameters are:

Phi:

 [0.62212826 0.17086601 0.20700573]

mu:

 [array([0.56593565, 0.37610181, 0.68064027, 0.68362035]), array([0.08333333, 0.5       , 0.06779661, 0.04166667]), array([0.13888889, 0.41666667, 0.06779661, 0.        ])]

sigma:

 [array([[0.03543998, 0.01463193, 0.02240035, 0.02065583],
       [0.01463193, 0.02020826, 0.01006346, 0.0135697 ],
       [0.02240035, 0.01006346, 0.01934445, 0.01999573],
       [0.02065583, 0.0135697 , 0.01999573, 0.02982374]]), array([[ 0.05580579, -0.00257483,  0.06258859,  0.06329843],
       [-0.00257483,  0.03238601, -0.02055817, -0.01894744],
       [ 0.06258859, -0.02055817,  0.09152202,  0.09476417],
       [ 0.06329843, -0.01894744,  0.09476417,  0.10468592]]), array([[ 0.05580579, -0.00257483,  0.06258859,  0.06329843],
       [-0.00257483,  0.03238601, -0.02055817, -0.01894744],
       [ 0.06258859, -0.02055817,  0.09152202,  0.09476417],
       [ 0.06329843, -0.01894744,  0.09476417,  0.10468592]])]

Test set accurac

In [6]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=k)
gmm.fit(X_train)

Y_pred = gmm.predict(X_test)

# Make sure that the clusters represented in both Y_test and Y_pred are the same
permutation = np.array([mode(Y_test[Y_pred == i]).mode.item() for i in range(k)])
permuted_prediction = permutation[Y_pred]

# Compute test set accuracy  
acc = accuracy_score(permuted_prediction, Y_test)
print("Test set accuracy: {:.2f}".format(acc))

Test set accuracy: 1.00


### References:
http://www.oranlooney.com/post/ml-from-scratch-part-5-gmm/
https://medium.com/@siddharthvadgama/gaussian-mixture-model-gmm-using-em-algorithm-from-scratch-6b7c764aac9f